Post-Implementation Improvements / Refactoring Suggestions
 After the current implementation, consider making the following changes to clean up and optimize the project:
 1. Modularization: Break the code into multiple files or at least define separate classes/modules for: Game logic (Snake, Food, Wall, Level) UI rendering Event handling 
 2. Save/Load Game Functionality Allow the player to pause and later resume their game. This could include: Saving the snake’s current position, direction, score, level Optionally store it in a JSON/CSV or txt.

In [3]:
import pygame
import sys
import random
import time
import json
import copy

pygame.init()

# Game settings
scale = 15
score = 0
level = 0
SPEED = 10

WIDTH, HEIGHT = 500, 500
font_colour = (255, 255, 255)
defeat_colour = (255, 0, 0)
background_top = (0, 0, 50)
background_bottom = (0, 0, 0)

# Display creation
display = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")
clock = pygame.time.Clock()

# background functions
def draw_background():
    display.fill(background_top)

# Text functions
def show_text(text, x, y, color, size=20):
    font = pygame.font.SysFont(None, size)
    surface = font.render(text, True, color)
    display.blit(surface, (x, y))

# Snake class
class Snake:
    def __init__(self, x_start, y_start, scale):
        self.scale = scale
        self.reset(x_start, y_start)

    def reset(self, x_start, y_start):
        self.x = x_start
        self.y = y_start
        self.x_dir = 1
        self.y_dir = 0
        self.history = [[self.x, self.y]]
        self.length = 1

    def handle_key(self, key):
        if self.y_dir == 0:
            if key == pygame.K_UP:
                self.x_dir = 0
                self.y_dir = -1
            elif key == pygame.K_DOWN:
                self.x_dir = 0
                self.y_dir = 1
        elif self.x_dir == 0:
            if key == pygame.K_LEFT:
                self.x_dir = -1
                self.y_dir = 0
            elif key == pygame.K_RIGHT:
                self.x_dir = 1
                self.y_dir = 0

    def show(self, display):
        for i in range(self.length):
            color = (255, 247, 0) if i == 0 else (255, 137, 0) 
            pygame.draw.rect(display, color, (*self.history[i], self.scale, self.scale))

    def update(self):
        for i in range(self.length - 1, 0, -1):
            self.history[i] = copy.deepcopy(self.history[i - 1])
        self.history[0][0] += self.x_dir * self.scale
        self.history[0][1] += self.y_dir * self.scale

    def grow(self):
        self.length += 1
        self.history.append(self.history[-1])

    def check_eaten(self, fx, fy):
        return abs(self.history[0][0] - fx) < self.scale and abs(self.history[0][1] - fy) < self.scale

    def death(self):
        for i in range(1, self.length):
            if self.history[0] == self.history[i] and self.length > 2:
                return True
        return False

    def hit_wall(self, width, height):
        x, y = self.history[0]
        return x < 0 or x >= width or y < 0 or y >= height

    def load(self, body, direction):
        self.history = body
        self.length = len(body)
        self.x_dir, self.y_dir = direction

# Food class
class Food:
    def __init__(self, width, height, scale):
        self.width = width
        self.height = height
        self.scale = scale
        self.color = (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))

    def new_location(self):
        self.x = random.randrange(1, self.width // self.scale - 1) * self.scale
        self.y = random.randrange(1, self.height // self.scale - 1) * self.scale

    def show(self, display):
        pygame.draw.rect(display, self.color, (self.x, self.y, self.scale, self.scale))

    def load(self, position):
        self.x, self.y = position

# Save and Load functions
def save_game(snake, food, score, level, speed):
    data = {
        'snake': snake.history,
        'direction': (snake.x_dir, snake.y_dir),
        'food': (food.x, food.y),
        'score': score,
        'level': level,
        'speed': speed
    }
    with open('savefile.json', 'w') as f:
        json.dump(data, f)

def load_game():
    with open('savefile.json', 'r') as f:
        return json.load(f)

# Main cycle
def game_loop():
    global score, level, SPEED

    snake = Snake(WIDTH / 2, HEIGHT / 2, scale)
    food = Food(WIDTH, HEIGHT, scale)
    food.new_location()

    paused = False

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    pygame.quit()
                    sys.exit()
                if event.key == pygame.K_p:
                    paused = not paused
                if event.key == pygame.K_s:
                    save_game(snake, food, score, level, SPEED)
                if event.key == pygame.K_l:
                    data = load_game()
                    snake.load(data['snake'], data['direction'])
                    food.load(data['food'])
                    score = data['score']
                    level = data['level']
                    SPEED = data['speed']
                snake.handle_key(event.key)

        if paused:
            continue

        draw_background()
        snake.show(display)
        snake.update()
        food.show(display)

        show_text(f"Score: {score}", 10, 10, font_colour)
        show_text(f"Level: {level}", 100, 10, font_colour)

        if snake.check_eaten(food.x, food.y):
            food.new_location()
            score += random.randint(1, 5)
            snake.grow()

        if snake.death() or snake.hit_wall(WIDTH, HEIGHT):
            score = 0
            level = 0
            show_text("Game Over!", 120, 200, defeat_colour, 60)
            pygame.display.update()
            time.sleep(3)
            snake.reset(WIDTH / 2, HEIGHT / 2)

        pygame.display.update()
        clock.tick(SPEED)

# Game run
if __name__ == '__main__':
    game_loop()


SystemExit: 